<a href="https://colab.research.google.com/github/elangbijak4/Time-Series-Research/blob/main/Modifikasi1_Testing_TSMIXER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perubahan dari TSMIXER, adalah penambahan mekanisme attention dan lapisan dropout menjadi spatialdroput1D. Pertama untuk mempertajam penyerapan pola, kedua untuk mengurangi mekanisme drop neuron yang berlebihan.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

def res_block(inputs, norm_type, activation, dropout, ff_dim):
    """Residual block of Enhanced TSMixer."""
    norm = (
        layers.LayerNormalization
        if norm_type == 'L'
        else layers.BatchNormalization
    )

    # Temporal Linear with Attention
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.MultiHeadAttention(num_heads=4, key_dim=x.shape[-1])(x, x)
    x = layers.Dense(x.shape[-1], activation=activation)(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.SpatialDropout1D(dropout)(x)
    res = x + inputs

    # Feature Linear
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation=activation)(
        x
    )  # [Batch, Input Length, FF_Dim]
    x = layers.SpatialDropout1D(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.SpatialDropout1D(dropout)(x)
    return x + res

def build_enhanced_model(input_shape, pred_len, norm_type, activation, n_block, dropout, ff_dim, target_slice):
    """Build Enhanced TSMixer model."""
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
        x = res_block(x, norm_type, activation, dropout, ff_dim)

    if target_slice:
        x = x[:, :, target_slice]

    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)

Demonstrasi penggunaan model yang dimodifikasi

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Res block as defined before
def res_block(inputs, norm_type, activation, dropout, ff_dim):
    norm = (
        layers.LayerNormalization
        if norm_type == 'L'
        else layers.BatchNormalization
    )

    # Temporal Linear with Attention
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.MultiHeadAttention(num_heads=4, key_dim=x.shape[-1])(x, x)
    x = layers.Dense(x.shape[-1], activation=activation)(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.SpatialDropout1D(dropout)(x)
    res = x + inputs

    # Feature Linear
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation=activation)(
        x
    )  # [Batch, Input Length, FF_Dim]
    x = layers.SpatialDropout1D(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.SpatialDropout1D(dropout)(x)
    return x + res

# Build enhanced model
def build_enhanced_model(input_shape, pred_len, norm_type, activation, n_block, dropout, ff_dim, target_slice):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
        x = res_block(x, norm_type, activation, dropout, ff_dim)

    if target_slice:
        x = x[:, :, target_slice]

    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel]

    return tf.keras.Model(inputs, outputs)

# Data preparation
batch_size = 64
input_length = 30
num_features = 10
pred_length = 5

# Generate dummy data
x_train = np.random.rand(batch_size, input_length, num_features)
y_train = np.random.rand(batch_size, pred_length, num_features)

# Define model parameters
input_shape = (input_length, num_features)
norm_type = 'L'
activation = 'relu'
n_block = 4
dropout = 0.1
ff_dim = 64
target_slice = None

# Build model
model = build_enhanced_model(input_shape, pred_length, norm_type, activation, n_block, dropout, ff_dim, target_slice)

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(x_train, y_train, epochs=10, batch_size=batch_size)

# Generate new data for inference
x_new = np.random.rand(batch_size, input_length, num_features)

# Predict
y_pred = model.predict(x_new)

# Print shapes
print(f"Input shape: {x_new.shape}")
print(f"Predicted output shape: {y_pred.shape}")

Epoch 1/10
1/1 [==============================] - 9s 9s/step - loss: 3.1950
Epoch 2/10
1/1 [==============================] - 0s 52ms/step - loss: 2.5864
Epoch 3/10
1/1 [==============================] - 0s 48ms/step - loss: 2.2545
Epoch 4/10
1/1 [==============================] - 0s 50ms/step - loss: 1.9724
Epoch 5/10
1/1 [==============================] - 0s 51ms/step - loss: 1.7330
Epoch 6/10
1/1 [==============================] - 0s 51ms/step - loss: 1.5469
Epoch 7/10
1/1 [==============================] - 0s 54ms/step - loss: 1.5023
Epoch 8/10
1/1 [==============================] - 0s 51ms/step - loss: 1.2797
Epoch 9/10
1/1 [==============================] - 0s 51ms/step - loss: 1.0967
Epoch 10/10
2/2 [==============================] - 1s 15ms/step
Input shape: (64, 30, 10)
Predicted output shape: (64, 5, 10)
